                                       特征筛选

In [1]:
#导入数据分析包
import numpy as np
import pandas as pd
import sklearn as sk
from os.path import dirname
import os
import sys


# 基础路径:即代码所在路径
BASEPATH=str(os.getcwd()).replace('\\','/')
print(BASEPATH)

C:/Users/85242/Desktop/特征选择打分


In [2]:
#导入数据集并且查看数据情况
f=open(BASEPATH+'/kdd99.csv')
df=pd.read_csv(f,low_memory=False)
print(df.head(5))

   1    2         3   4    5     6  7  8  9  10       ...           33    34  \
0  0  tcp  ftp_data  SF  615     0  0  0  0   0       ...         56.0  0.32   
1  0  tcp      http  SF  312   677  0  0  0   0       ...        255.0  1.00   
2  0  udp   private  SF  105   146  0  0  0   0       ...        254.0  1.00   
3  0  tcp      http  SF  307  1528  0  0  0   0       ...        255.0  1.00   
4  0  udp   private  SF  105   146  0  0  0   0       ...        254.0  1.00   

     35    36    37   38   39   40   41              42  
0  0.03  0.32  0.00  0.0  0.0  0.0  0.0         normal.  
1  0.00  0.05  0.04  0.0  0.0  0.0  0.0         normal.  
2  0.01  0.00  0.00  0.0  0.0  0.0  0.0         normal.  
3  0.00  0.00  0.00  0.0  0.0  0.0  0.0         normal.  
4  0.01  0.01  0.00  0.0  0.0  0.0  0.0  snmpgetattack.  

[5 rows x 42 columns]


In [3]:
#查看标签数据有哪些类别，且做数值转换
labe=df['42']
category = pd.Categorical(labe)
df['42']=category.labels

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: 'labels' is deprecated. Use 'codes' instead
  after removing the cwd from sys.path.


In [4]:
df1=df.select_dtypes(include=['object'])
print(df1.head(4))

     2         3   4 15
0  tcp  ftp_data  SF  0
1  tcp      http  SF  0
2  udp   private  SF  0
3  tcp      http  SF  0


In [5]:
#此部分为数据预处理部分
df1=df.select_dtypes(include=['object'])
print(df1.head(5))
colums=df1.columns
print(colums)
for colum in colums:
    
    if int(colum)<=15:
        labe=df1[colum]
        category = pd.Categorical(labe)
        df1[colum]=category.labels
    else:
        df1[colum]=df1[colum].replace('?',0)
        df1[colum]=df1[colum].astype(np.float64)

#删除object数据类型        
for colum in colums:
    df.drop(colum,axis=1, inplace=True)
        
df=pd.concat([df1,df],axis=1)
print(df.head(5))
        

     2         3   4 15
0  tcp  ftp_data  SF  0
1  tcp      http  SF  0
2  udp   private  SF  0
3  tcp      http  SF  0
4  udp   private  SF  0
Index(['2', '3', '4', '15'], dtype='object')


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: 'labels' is deprecated. Use 'codes' instead
  # This is added back by InteractiveShellApp.init_path()
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


   2   3   4  15  1    5     6  7  8  9 ...     33    34    35    36    37  \
0  2  20  10   0  0  615     0  0  0  0 ...   56.0  0.32  0.03  0.32  0.00   
1  2  23  10   0  0  312   677  0  0  0 ...  255.0  1.00  0.00  0.05  0.04   
2  3  47  10   0  0  105   146  0  0  0 ...  254.0  1.00  0.01  0.00  0.00   
3  2  23  10   0  0  307  1528  0  0  0 ...  255.0  1.00  0.00  0.00  0.00   
4  3  47  10   0  0  105   146  0  0  0 ...  254.0  1.00  0.01  0.01  0.00   

    38   39   40   41  42  
0  0.0  0.0  0.0  0.0  16  
1  0.0  0.0  0.0  0.0  16  
2  0.0  0.0  0.0  0.0  16  
3  0.0  0.0  0.0  0.0  16  
4  0.0  0.0  0.0  0.0  28  

[5 rows x 42 columns]


In [42]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score  
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
#保存结果
file_out=open(BASEPATH+'/result4.csv',mode='w')
def save_result(rfr,x,y,type,cnt):
    
    if type==1:
        a=accuracy_score(y, rfr.predict(x) )
        b=recall_score(y, rfr.predict(x),average='micro')
        c=f1_score(y, rfr.predict(x),average='micro')
        d=classification_report(y, rfr.predict(x))
        aline=format('%s,%s,%s,%s,%s'%('信息熵',cnt,a,b,c))
        print(aline)
        file_out.write(aline+'\n')
        file_out.flush()
        
    else:    
        a=accuracy_score(y, rfr.predict(x) )
        b=recall_score(y, rfr.predict(x),average='micro')
        c=f1_score(y, rfr.predict(x),average='micro')
        d=classification_report(y, rfr.predict(x))
        aline=format('%s,%s,%s,%s,%s'%('fiter',cnt,a,b,c))
        print(aline)
        file_out.write(aline+'\n')
        file_out.flush() 
    
        
        

In [43]:
#选择RF（随机森林）建立模型，并且对特征进行打分。随机森林打分的最终落脚点为CART树的基尼指数(基尼指数越高代表信息越确定)，跟熵的性质差不多，熵是衡量信息的不确定性
from sklearn.utils import shuffle
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split  
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import Imputer

names=list(df.columns)
df = Imputer().fit_transform(df)
df=pd.DataFrame(df,columns=names)



#打乱数据集
df=shuffle(df)
#获取x,y
x= df.iloc[:,:41] 
y=df.iloc[:,41:42]

rfr = RandomForestClassifier(random_state=0, n_estimators=2000, n_jobs=-1)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=0) 

names=list(x.columns)

clf = ExtraTreesClassifier(criterion='entropy')
X_new = clf.fit(x, y)
names=sorted(zip(map(lambda x: round(x, 4), clf.feature_importances_), names),reverse=True)

list_names=[]
for row in range(len(names)):
        list_names.append(names[row][1])
        if ((row%10==0) or (row==247)) and (row!=0):
           #信息熵训练模型
              row=row+1
              x_train1=x_train[list_names]
              x_test1=x_test[list_names]
              print(x_test1.shape)   
              rfr.fit(x_train1,y_train)   
              save_result(rfr,x_test1,y_test,1,row)
                
            #卡方验证输出top前N特征
            
              X_new = SelectKBest(chi2, k=row).fit_transform(x_train, y_train)  
              X_new1 = SelectKBest(chi2, k=row).fit_transform(x_test, y_test)
              print(X_new1.shape)
              rfr.fit(X_new,y_train)
              save_result(rfr,X_new1,y_test,2,row)



C:\Anaconda\lib\site-packages\ipykernel_launcher.py:29: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


(58752, 11)


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:41: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


信息熵,11,0.9717967047930284,0.9717967047930284,0.9717967047930284
(58752, 11)


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:48: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:41: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


...

C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


信息熵,41,0.9797623910675382,0.9797623910675382,0.9797623910675382
(58752, 41)


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:48: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


fiter,41,0.9797623910675382,0.9797623910675382,0.9797623910675382


C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
X_new = SelectKBest(chi2, k=5).fit_transform(x_train, y_train)  
print(X_new)

[[ 399.    0.    0.  399.  399.]
 [ 399.    0.    0.  399.  399.]
 [ 399.    0.    0.  399.  399.]
 ...
 [ 433.    0.    0.  433.  433.]
 [ 400.    0.    0.  400.  400.]
 [1639.    0.    0. 1639. 1639.]]


In [8]:
from sklearn.utils import shuffle
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split  
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import Imputer
#打乱数据集

names=list(df.columns)
df = Imputer().fit_transform(df)
df=pd.DataFrame(df,columns=names)



df=shuffle(df)
#获取x,y
x= df.iloc[:,:41] 
y=df.iloc[:,41:42]

rfr = RandomForestClassifier(random_state=0, n_estimators=2000, n_jobs=-1)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=0) 

names=list(x.columns)

clf = ExtraTreesClassifier(criterion='entropy')
X_new = clf.fit(x, y)
names=sorted(zip(map(lambda x: round(x, 4), clf.feature_importances_), names),reverse=True)
print('熵特征排序结果：')
print(names)

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


熵特征排序结果：
[(0.2187, '36'), (0.0869, '2'), (0.0854, '4'), (0.0832, '23'), (0.06, '27'), (0.0565, '24'), (0.05, '12'), (0.0422, '3'), (0.0404, '40'), (0.0384, '41'), (0.0376, '33'), (0.0344, '29'), (0.0259, '28'), (0.0221, '35'), (0.021, '32'), (0.0182, '31'), (0.0148, '34'), (0.014, '30'), (0.01, '5'), (0.008, '6'), (0.0053, '25'), (0.0047, '37'), (0.0045, '1'), (0.0044, '38'), (0.0029, '11'), (0.0028, '22'), (0.0027, '26'), (0.0026, '39'), (0.001, '10'), (0.0004, '13'), (0.0002, '8'), (0.0002, '14'), (0.0001, '9'), (0.0001, '21'), (0.0001, '19'), (0.0001, '18'), (0.0001, '17'), (0.0001, '16'), (0.0, '7'), (0.0, '20'), (0.0, '15')]


In [12]:
#保存模型
from sklearn.externals import joblib
joblib.dump(rfr_r, BASEPATH+'/rfr_r.model')
#加载模型
#rfr_r = joblib.load(BASEPATH+'/rfr_r.model')

NameError: name 'rfr_r' is not defined

In [28]:
# 模型评估
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score  
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
# 测试集评估
print('测试集评估:')
print(classification_report(y_test, rfr.predict(x_test) ))

# 预测集评估
print('预测集评估:')
print(accuracy_score(y_test, rfr.predict(x_test) ))
print(recall_score(y_test, rfr.predict(x_test),average='micro'))
print(f1_score(y_test, rfr.predict(x_test),average='micro'))


测试集评估:
             precision    recall  f1-score   support

          0       1.00      0.80      0.89       133
          1       1.00      0.40      0.57         5
          2       0.94      0.80      0.86        20
          3       1.00      1.00      1.00        25
          4       0.87      0.99      0.93       418
          6       1.00      0.67      0.80         9
          7       0.99      1.00      1.00      1114
          8       0.94      0.89      0.92        74
          9       1.00      0.98      0.99        85
         10       1.00      1.00      1.00     17992

avg / total       0.99      0.99      0.99     19875

预测集评估:
0.9941635220125786
0.9941635220125786
0.9941635220125786
